In [1]:
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import pandas as pd

import lightning.pytorch as pl


from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer, MultiNormalizer, TorchNormalizer
from pytorch_forecasting.metrics import MAE, QuantileLoss

from TFT_modeling import GroupFairnessMAE, baseline_prediction, train_model

In [3]:
df_ryiadh = pd.read_csv("../../possible_datasets/KSMC_Hospital/data_preprocessed.csv")
df_ryiadh["Admission_Date"] = pd.to_datetime(df_ryiadh["Admission_Date"])

# add time index
df_ryiadh["time_idx"] = df_ryiadh["Admission_Date"].dt.year * 12 + df_ryiadh["Admission_Date"].dt.month
df_ryiadh["time_idx"] -= df_ryiadh["time_idx"].min()

# add additional features
df_ryiadh["Admission_month"] = df_ryiadh.Admission_Date.dt.month.astype(str).astype(
    "category"
)
df_ryiadh["Admission_year"] = df_ryiadh.Admission_Date.dt.year.astype(float)
df_ryiadh["Admission_Counts"] = df_ryiadh["Admission_Counts"].astype(float)
df_ryiadh.drop(columns=["Admission_Date"], inplace=True)
df_ryiadh["Age"] = df_ryiadh["Age"].apply(lambda x: 0. if x == '0-17' else 1. if x == '18-45' else 2. if x == '46-65' else 3)
df_ryiadh

,Age,Gender,Nationality,Hospital_Name,Admission_Counts,time_idx,Admission_month,Admission_year
0,0.0,Female,Non-Saudi,King Abdulaziz Medical City,6.0,0,1,2018.0
1,0.0,Female,Non-Saudi,King Abdulaziz Medical City,7.0,1,2,2018.0
2,0.0,Female,Non-Saudi,King Abdulaziz Medical City,2.0,2,3,2018.0
3,0.0,Female,Non-Saudi,King Abdulaziz Medical City,5.0,3,4,2018.0
4,0.0,Female,Non-Saudi,King Abdulaziz Medical City,3.0,4,5,2018.0
...,...,...,...,...,...,...,...,...
4795,3.0,Male,Saudi,King Saud Medical City,23.0,70,11,2023.0
4796,3.0,Male,Saudi,King Saud Medical City,31.0,71,12,2023.0
4797,3.0,Male,Saudi,King Saud Medical City,31.0,72,1,2024.0
4798,3.0,Male,Saudi,King Saud Medical City,24.0,73,2,2024.0


In [4]:
max_prediction_length = 6
max_encoder_length = 24
training_cutoff = df_ryiadh["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df_ryiadh[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Admission_Counts",
    group_ids=["Age", "Gender", "Nationality", "Hospital_Name"],  #groups of time series in my dataset
    min_encoder_length=max_encoder_length
    // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Gender", "Nationality", "Hospital_Name"],
    static_reals=["Age"],
    time_varying_known_categoricals=["Admission_month"],
    time_varying_known_reals=["Admission_year"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "Admission_Counts",
    ],
    target_normalizer=GroupNormalizer(
        groups=["Age", "Gender", "Nationality", "Hospital_Name"], transformation="softplus"
    ),  # use softplus and normalize by group for target variable, helps for more stable training and strongly differing scales
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(
    training, df_ryiadh, predict=True, stop_randomization=True
)

# create dataloaders for model
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=0
)

In [5]:
import torch

y_true  = torch.tensor([10., 8., 12., 5., 7., 9.])
y_pred  = torch.tensor([13., 6., 11., 6., 9., 8.])
groups  = torch.tensor([0,0,0,1,1,1])
GroupFairnessMAE(groups)(y_pred, y_true)

tensor(0.2222)

In [6]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
time_len = val_dataloader.dataset.data["time"].max() + 1
groups_ryiadh = val_dataloader.dataset.data["groups"][::time_len]
baseline_prediction(val_dataloader, GroupFairnessMAE(groups_ryiadh))

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


tensor(0.9122)

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="cpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training, 
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    loss=GroupFairnessMAE(groups_ryiadh),
    optimizer="adam", #ranger
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
    logging_metrics=[MAE(), GroupFairnessMAE(groups_ryiadh)],
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")

Seed set to 42
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Number of parameters in network: 7.7k


In [8]:
time_len = train_dataloader.dataset.data["time"].max() + 1
groups_ryiadh = train_dataloader.dataset.data["groups"][::time_len].repeat_interleave(6, dim=0)

In [10]:
trainer = train_model(tft, trainer, train_dataloader, val_dataloader, training, GroupFairnessMAE(groups_ryiadh), [MAE(), GroupFairnessMAE(groups_ryiadh)])

Restoring states from the checkpoint path at c:\Users\Luca\Studium\Master\Master-project\backup_and_old_code\TFT\.lr_find_db4a4eb7-ef3b-4b26-9ea6-bcd75c013e9c.ckpt
Restored all states from the checkpoint at c:\Users\Luca\Studium\Master\Master-project\backup_and_old_code\TFT\.lr_find_db4a4eb7-ef3b-4b26-9ea6-bcd75c013e9c.ckpt


IndexError: The shape of the mask [64] at index 0 does not match the shape of the indexed tensor [185] at index 0

In [9]:
metrics = trainer.logged_metrics
print(metrics)

{'train_loss_step': tensor(3.1533), 'val_loss': tensor(2.4096), 'val_MAE': tensor(3.0386), 'val_TorchMetricWrapper': tensor(0.5262), 'train_loss_epoch': tensor(2.8499)}


In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3376), started 2:56:17 ago. (Use '!kill 3376' to kill it.)

Code template: https://pytorch-forecasting.readthedocs.io/en/v1.4.0/tutorials/stallion.html